In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the csv from the WeatherPy into a pandas dataframe
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cidreira,-30.1811,-50.2056,26.58,78,1,5.75,BR,"{'coord': {'lon': -50.2056, 'lat': -30.1811}, ..."
1,1,longyearbyen,78.2186,15.6401,3.91,60,40,1.54,SJ,"{'coord': {'lon': 15.6401, 'lat': 78.2186}, 'w..."
2,2,berezovyy,51.6667,135.7000,-21.14,99,100,1.19,RU,"{'coord': {'lon': 135.7, 'lat': 51.6667}, 'wea..."
3,3,port mathurin,-19.6833,63.4167,26.95,86,99,6.24,MU,"{'coord': {'lon': 63.4167, 'lat': -19.6833}, '..."
4,4,edinburgh of the seven seas,-37.0676,-12.3116,18.55,88,100,12.18,SH,"{'coord': {'lon': -12.3116, 'lat': -37.0676}, ..."


In [11]:
# Ensure dataframe has required columns
city_data_df = city_data_df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"})

# Plot the interactive map with OpenStreetMap (OSM) tiles
city_data_df.hvplot.points(
    x="longitude",
    y="latitude",
    geo=True,  # Enables geographic mapping
    tiles="OSM",  # Use OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    color="Max Temp",  # Color points by max temperature
    cmap="coolwarm",  # Use a colormap for better visualization
    size="Humidity",  # Size of points based on humidity levels
    scale=0.6,  # Adjust scaling factor (to keep points visible but proportional)
    hover_cols=["City", "Country", "Max Temp", "Humidity"]  # Show extra details on hover
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (Max Temp,Humidity,City,Country)

In [5]:
# Filter the dataframe based on the specified conditions 
filtered_city_data_df = city_data_df[(city_data_df["Max Temp"]>21)&
                                     (city_data_df["Max Temp"]<27)&
                                     (city_data_df["Wind Speed"]<4.5)&
                                     (city_data_df["Cloudiness"] == 0)
                                     ]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample results
filtered_city_data_df.head()

,City_ID,City,latitude,longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hover_text
51,51,aswan,24.0934,32.9070,22.61,28,0,3.09,EG,"{'coord': {'lon': 32.907, 'lat': 24.0934}, 'we...","aswan, EG"
96,96,guerrero negro,27.9769,-114.0611,22.02,42,0,1.43,MX,"{'coord': {'lon': -114.0611, 'lat': 27.9769}, ...","guerrero negro, MX"
125,125,cabo san lucas,22.8909,-109.9124,23.17,56,0,1.03,MX,"{'coord': {'lon': -109.9124, 'lat': 22.8909}, ...","cabo san lucas, MX"
172,172,bafoulabe,13.8065,-10.8321,25.92,8,0,2.86,ML,"{'coord': {'lon': -10.8321, 'lat': 13.8065}, '...","bafoulabe, ML"
181,181,tha khanon,9.0302,98.9529,21.33,84,0,0.32,TH,"{'coord': {'lon': 98.9529, 'lat': 9.0302}, 'we...","tha khanon, TH"


In [6]:

# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[["City", "Country", "latitude", "longitude", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,latitude,longitude,Humidity,Hotel Name
51,aswan,EG,24.0934,32.9070,28,
96,guerrero negro,MX,27.9769,-114.0611,42,
125,cabo san lucas,MX,22.8909,-109.9124,56,
172,bafoulabe,ML,13.8065,-10.8321,8,
181,tha khanon,TH,9.0302,98.9529,84,


In [8]:
# Set parameters to search for a hotel
hotel_names = []
radius = 10000
category = "accommodation.hotel"

# Loop through each city in the dataframe
params = {
    'categories': category,
    'apiKey': geoapify_key    
}

# Print message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df dataframe
for index, row in hotel_df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    print(f"adding {lat}, {lon} to parameters")

    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lon
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)
    
    # Check if the api response is successful
    if name_address.status_code == 200:
        # Convert the API response to JSON format
        name_address = name_address.json()
        
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    else:
        print(f"Error with API request for {hotel_df.loc[index, 'City']}: {name_address.status_code}")

# Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adding 24.0934, 32.907 to parameters
aswan - nearest hotel: Oscar Hotel
adding 27.9769, -114.0611 to parameters
guerrero negro - nearest hotel: Plaza sal paraiso
adding 22.8909, -109.9124 to parameters
cabo san lucas - nearest hotel: Comfort Rooms
adding 13.8065, -10.8321 to parameters
bafoulabe - nearest hotel: No hotel found
adding 9.0302, 98.9529 to parameters
tha khanon - nearest hotel: No hotel found
adding 30.1766, -85.8055 to parameters
panama city beach - nearest hotel: Shores of Panama
adding 11.747, 11.9608 to parameters
damaturu - nearest hotel: No hotel found
adding -34.7333, 135.8667 to parameters
port lincoln - nearest hotel: Boston Hotel
adding 28.017, -97.5169 to parameters
san patricio - nearest hotel: No hotel found
adding 17.7229, 42.2526 to parameters
ad darb - nearest hotel: No hotel found
adding 13.2114, 12.0241 to parameters
maine soroa - nearest hotel: No hotel found
adding 20.7, -105.2 to parameters
ixtapa - nearest hotel: Ma. Cristina


,City,Country,latitude,longitude,Humidity,Hotel Name
51,aswan,EG,24.0934,32.9070,28,Oscar Hotel
96,guerrero negro,MX,27.9769,-114.0611,42,Plaza sal paraiso
125,cabo san lucas,MX,22.8909,-109.9124,56,Comfort Rooms
172,bafoulabe,ML,13.8065,-10.8321,8,No hotel found
181,tha khanon,TH,9.0302,98.9529,84,No hotel found
191,panama city beach,US,30.1766,-85.8055,63,Shores of Panama
195,damaturu,NG,11.7470,11.9608,17,No hotel found
230,port lincoln,AU,-34.7333,135.8667,66,Boston Hotel
312,san patricio,US,28.0170,-97.5169,40,No hotel found
314,ad darb,SA,17.7229,42.2526,70,No hotel found


In [24]:
# Assign a unique color to each city (or hotel name)
hotel_df["color"] = hotel_df["City"].astype("category").cat.codes  # Convert City names to numerical categories

# Plot the interactive map with OpenStreetMap (OSM) tiles
hotel_df.hvplot.points(
    x="longitude",
    y="latitude",
    geo=True,  # Enables geographic mapping
    tiles="OSM",  # Use OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    color="City",  # Color points by max temperature
    cmap="Category10",  # Use a colormap for better visualization
    size="Humidity",  # Size of points based on humidity levels
    scale=.8,  # Adjust scaling factor (to keep points visible but proportional)
    hover_cols=["Hotel Name", "City", "Country", "Humidity"],  # Show extra details on hover
    legend ="right" #place the legend to the right
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,Hotel Name,Country)